In [29]:
import pandas as pd

In [30]:
df = pd.read_excel('cashback.xlsx')
#df = df[df['возраст'] >= 18]
#23
expenses_cats = list(df.columns.values)[6:23]
cats = [c.replace('оборот_', '') for c in expenses_cats]

#na_count = 0
#for name in cats:
#    diff = df['активация_кэшбэка_'+str(name)].isnull().sum()
#    na_count += diff
#    print(f'{name}: {diff}')
#print('всего:', na_count)

na_count = 0
for name in cats:
    diff = df['активация_кэшбэка_'+str(name)][df['активация_кэшбэка_'+str(name)] == 1].count()
    na_count += diff
    print(f'1 в {name}: {diff}')
print('всего 1:', na_count)

na_count = 0
for name in cats:
    diff = df['активация_кэшбэка_'+str(name)][df['активация_кэшбэка_'+str(name)] == 0].count()
    na_count += diff
    print(f'0 в {name}: {diff}')
print('всего 0:', na_count)
    
df[['активация_кэшбэка_'+str(name) for name in cats]] = df[['активация_кэшбэка_'+str(name) for name in cats]].fillna(0)
df[expenses_cats] = df[expenses_cats].fillna(0)

1 в аптеки: 26929
1 в рестораны: 17202
1 в одежда_и_обувь: 16238
1 в автоуслуги: 7404
1 в супермаркеты: 8426
1 в такси: 6947
1 в красота: 11769
1 в развлечения: 14613
1 в жд_билеты: 6470
1 в образование: 2691
1 в дом_и_ремонт: 1419
1 в спорттовары: 5509
1 в животные: 4637
1 в цветы: 4849
1 в фастфуд: 226
1 в каршеринг: 425
1 в аренда_авто: 358
всего 1: 136112
0 в аптеки: 12313
0 в рестораны: 2318
0 в одежда_и_обувь: 3105
0 в автоуслуги: 4553
0 в супермаркеты: 289
0 в такси: 1607
0 в красота: 9449
0 в развлечения: 9932
0 в жд_билеты: 13051
0 в образование: 8706
0 в дом_и_ремонт: 978
0 в спорттовары: 7868
0 в животные: 3563
0 в цветы: 12953
0 в фастфуд: 431
0 в каршеринг: 1466
0 в аренда_авто: 1064
всего 0: 93646


In [31]:
countries = df.groupby('ключ_клиента')['регион_проживания'].first()
columns = {'регион': countries}

for c in cats:
    #df[df['активация_кэшбэка_'+c] == 1].groupby('ключ_клиента')['оборот_'+c].sum() != 0
    is_not_empty = df.groupby('ключ_клиента')['оборот_'+c].sum() != 0
    print(c, ':', is_not_empty[is_not_empty.values == False].count())
    with_cashback = df[df['активация_кэшбэка_'+c] == 1].groupby('ключ_клиента')['оборот_'+c].mean()[is_not_empty]
    #print(c+':', df[df['активация_кэшбэка_'+c] == 1].groupby('ключ_клиента')['оборот_'+c].mean().count() - with_cashback.count())
    without_cashback = df[df['активация_кэшбэка_'+c] != 1].groupby('ключ_клиента')['оборот_'+c].mean()[is_not_empty]
    #print(c+':', df[df['активация_кэшбэка_'+c] != 1].groupby('ключ_клиента')['оборот_'+c].mean().count() - without_cashback.count())
    summ = (with_cashback + without_cashback) / 100
    columns['оборот_'+c+'_с_активацией'] = with_cashback / summ
    columns['оборот_'+c+'_без_активации'] = without_cashback / summ
    columns['повышение_оборота_'+c] = 0

active = pd.DataFrame(columns).fillna(0).groupby('регион').mean()

print('-----------------------------------------------------------------------------------------------------------------------')
na_count = 0
for c in cats:
    summ = (active['оборот_'+c+'_с_активацией'] + active['оборот_'+c+'_без_активации']) / 100
    
    diff1 = (active['оборот_'+c+'_с_активацией'] / summ).isnull().sum()
    diff2 = (active['оборот_'+c+'_без_активации'] / summ).isnull().sum()
    na_count += diff1 + diff2
    print(f'{c} с активацией: {diff1}')
    print(f'{c} без активации: {diff2}')
    print(active)
    #active['оборот_'+c+'_с_активацией'] = (active['оборот_'+c+'_с_активацией'] / summ).fillna((active['оборот_'+c+'_с_активацией'] / summ).dropna().mean())
    #active['оборот_'+c+'_без_активации'] = (active['оборот_'+c+'_без_активации'] / summ).fillna((active['оборот_'+c+'_без_активации'] / summ).dropna().mean())
    active['оборот_'+c+'_с_активацией'] = (active['оборот_'+c+'_с_активацией'] / summ).fillna(0)
    active['оборот_'+c+'_без_активации'] = (active['оборот_'+c+'_без_активации'] / summ).fillna(0)
    active['повышение_оборота_'+c] = active.apply(lambda l:  1 if l['оборот_'+c+'_с_активацией'] - l['оборот_'+c+'_без_активации'] > 5 else 0, axis=1)

print('всего:', na_count)

#active.to_excel('ds12.xlsx')

аптеки : 808
рестораны : 854
одежда_и_обувь : 993
автоуслуги : 4563
супермаркеты : 478
такси : 2155
красота : 3222
развлечения : 1607
жд_билеты : 6048
образование : 5726
дом_и_ремонт : 1234
спорттовары : 4780
животные : 5841
цветы : 3455
фастфуд : 587
каршеринг : 8415
аренда_авто : 9642
-----------------------------------------------------------------------------------------------------------------------
аптеки с активацией: 1
аптеки без активации: 1
                                   оборот_аптеки_с_активацией  \
регион                                                          
Алтайский край                                      47.106246   
Амурская область                                    45.385856   
Архангельская область                               45.130462   
Астраханская область                                52.327728   
Белгородская область                                40.516165   
...                                                       ...   
Хабаровский край         

In [32]:
countries = df.groupby('ключ_клиента')['регион_проживания'].first()
columns = {'регион': countries}

for c in cats:
    is_not_empty = df.groupby('ключ_клиента')['оборот_'+c].sum() != 0
    #is_not_empty_c = df[df['активация_кэшбэка_'+c] == 1].groupby('ключ_клиента')['оборот_'+c].sum() != 0
    #is_not_empty_nc = df[df['активация_кэшбэка_'+c] != 1].groupby('ключ_клиента')['оборот_'+c].sum() != 0
    #print(c, ':', is_not_empty[is_not_empty.values == False].count())
    with_cashback = df[df['активация_кэшбэка_'+c] == 1].groupby('ключ_клиента')['оборот_'+c].mean()[is_not_empty]
    without_cashback = df[df['активация_кэшбэка_'+c] != 1].groupby('ключ_клиента')['оборот_'+c].mean()[is_not_empty]
    columns['оборот_'+c+'_с_активацией'] = with_cashback
    columns['оборот_'+c+'_без_активации'] = without_cashback

abs_prev = pd.DataFrame(columns)
abs_prev[columns == 0] = pd.NA
#ccats = ['оборот_'+c+'_с_активацией' for c in cats]+['оборот_'+c+'_без_активации' for c in cats]
absolute = abs_prev.groupby('регион').median()
absolute.to_excel('absolute.xlsx')

abs_msc = absolute[absolute.index == 'Москва'].transpose()
abs_regs = absolute[absolute.index != 'Москва'].transpose()
#(is_not_empty_c and is_not_empty_nc).to_excel('bred.xlsx')


for i in range(34):
    row = abs_regs.iloc[i]
    mean = row[row.values != 0].median()
    abs_regs.iloc[i][abs_regs.iloc[i].values == 0] = mean

abs_regs = abs_regs.median(axis=1).to_frame().rename({0: 'Регионы'}, axis=1)

ccs = []
#1-46
for i in range(0, 33, 2):
    ccs.append(absolute.columns[i].replace('оборот_', '').replace('_с_активацией', ' с активацией').replace('_', ' '))
    ccs.append(absolute.columns[i+1].replace('оборот_', '').replace('_без_активации', ' без активации').replace('_', ' '))

res = pd.concat([abs_msc, abs_regs], axis=1)
#res = res.rename({27: 'Москва', 0: 'Регионы'}, axis=1)

c:\Users\Salir\Desktop\DANO_FINAL\venv\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning:

Mean of empty slice

c:\Users\Salir\Desktop\DANO_FINAL\venv\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning:

Mean of empty slice

c:\Users\Salir\Desktop\DANO_FINAL\venv\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning:

Mean of empty slice

c:\Users\Salir\Desktop\DANO_FINAL\venv\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning:

Mean of empty slice

c:\Users\Salir\Desktop\DANO_FINAL\venv\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning:

Mean of empty slice

c:\Users\Salir\Desktop\DANO_FINAL\venv\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning:

Mean of empty slice

c:\Users\Salir\Desktop\DANO_FINAL\venv\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning:

Mean of empty slice

c:\Users\Salir\Desktop\DANO_FINAL\venv\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning:

Mean 

In [24]:
res.T.to_excel('abs_sort.xlsx')

In [34]:
msc_abs = abs_msc.transpose()
regs_abs = abs_regs.transpose()

msc_diffs = pd.DataFrame()
regs_diffs = pd.DataFrame()

ccs2 = ['оборот_'+c.replace(' ', '_') for c in ccs]

for i in range(0, 33, 2):
    msc_diffs['разница_оборота_'+cats[i // 2]] = (msc_abs[ccs2[i]] / msc_abs[ccs2[i+1]] - 1) * 100
    regs_diffs['разница_оборота_'+cats[i // 2]] = (regs_abs[ccs2[i]] / regs_abs[ccs2[i+1]] - 1) * 100
    #msc_diffs['разница_оборота_'+cats[i // 2]] = (msc_abs[ccs2[i]] - msc_abs[ccs2[i+1]]) / (msc_abs[ccs2[i]] + msc_abs[ccs2[i+1]]) * 100
    #regs_diffs['разница_оборота_'+cats[i // 2]] = (regs_abs[ccs2[i]] - regs_abs[ccs2[i+1]]) / (regs_abs[ccs2[i]] + regs_abs[ccs2[i+1]]) * 100

diffs = pd.concat([msc_diffs, regs_diffs]).transpose()
diffs = diffs.drop('разница_оборота_каршеринг', axis=0)
diffs = diffs.drop('разница_оборота_аренда_авто', axis=0)
diffs

,Москва,Регионы
разница_оборота_аптеки,54.853526,61.515405
разница_оборота_рестораны,44.36929,52.495715
разница_оборота_одежда_и_обувь,63.611073,74.05167
разница_оборота_автоуслуги,157.945111,295.694109
разница_оборота_супермаркеты,19.494486,30.781902
разница_оборота_такси,88.887683,195.843436
разница_оборота_красота,63.287037,97.058824
разница_оборота_развлечения,40.300231,24.849023
разница_оборота_жд_билеты,256.887357,373.079536
разница_оборота_образование,412.04233,758.363985


In [37]:
import plotly.express as px
import plotly.graph_objects as go

# ['<br>'.join(list(c.replace('_', ' ')))
fig = go.Figure()
bar1 = px.bar(diffs, x=[c.replace('_', '<br>') for c in cats[:15]], y='Москва', text_auto='.3s').data[0]
bar2 = px.bar(diffs, x=[c.replace('_', '<br>') for c in cats[:15]], y='Регионы', text_auto='.3s').data[0]
bar1.update(marker=dict(color=['#F9B810'  for _ in range(17)]), name='Москва')
bar2.update(marker=dict(color=['#6C7074' for _ in range(17)]), name='Регионы')
fig.add_trace(bar1)
fig.add_trace(bar2)
fig.update_traces(textposition='outside', showlegend=True)
#fig.update_annotations(textangle=90, yshift=-40)
fig.update_xaxes(tickangle=0, showgrid=True, gridwidth=1, gridcolor='#a6a6a6')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#a6a6a6')
fig.update_layout(width=1500, height=600, font=dict(size=12), paper_bgcolor='#F7FBFD', plot_bgcolor='#F7FBFD', title='Разница оборота до и после активации кэшбэка в %')
fig.write_image('diffs.svg')
fig.show()

In [36]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

figs = []
for i in range(0, 33, 2):
    fig = make_subplots(rows=1, cols=2, column_titles=['Москва', 'Регионы'], y_title='Оборот')
    bar1 =px.bar(y=[res.iloc[i]['Москва'], res.iloc[i+1]['Москва']], x=[ccs[i], ccs[i+1]], text_auto='.4s')
    bar2 = px.bar(y=[res.iloc[i]['Регионы'], res.iloc[i+1]['Регионы']], x=[ccs[i], ccs[i+1]], text_auto='.4s')
    fig.add_trace(bar1.data[0], row=1, col=1)
    fig.add_trace(bar2.data[0], row=1, col=2)
    fig.update_traces(showlegend=False, textangle=0, marker=dict(color=['#F9B810', '#6C7074']))
    #fig.update_layout(barmode='relative')
    #fig.update_yaxes(title_text="Оборот %")
    figs.append(fig)
    colss = [res.iloc[i]['Москва'], res.iloc[i+1]['Москва'], res.iloc[i]['Регионы'], res.iloc[i+1]['Регионы']]
    colss = [round(cc / 1000) * 1000 + 1000 for cc in colss]
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#a6a6a6', range=[0, max(colss)])
    fig.update_layout(width=1200, height=600, font=dict(size=14), paper_bgcolor='#F7FBFD', plot_bgcolor='#F7FBFD')
    #msc_fig = px.bar(y=[res.iloc[i]['Москва'], res.iloc[i+1]['Москва']], x=[ccs[i], ccs[i+1]])
    #regs_fig = px.bar(y=[res.iloc[i]['Регионы'], res.iloc[i+1]['Регионы']], x=[ccs[i], ccs[i+1]])
    #figs.append(msc_fig)
    #figs.append(regs_fig)
#figs[0].write_image('test.svg')
for i in range(len(figs)): figs[i].write_image(f'./graphics7/{cats[i]}.svg')
for fig in figs: fig.show()

In [8]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

act = active.copy()
for c in cats:
    act = act.drop('повышение_оборота_'+c, axis=1)

msc = act[act.index == 'Москва'].transpose()
regs =act[act.index != 'Москва'].transpose()

for i in range(34):
    row = regs.iloc[i]
    mean = row[row.values != 0].mean()
    regs.iloc[i][regs.iloc[i].values == 0] = mean

regs = regs.mean(axis=1).to_frame().rename({0: 'Регионы'}, axis=1)

ccs = []
#1-46
for i in range(0, 33, 2):
    ccs.append(act.columns[i].replace('оборот_', '').replace('_с_активацией', ' с активацией').replace('_', ' '))
    ccs.append(act.columns[i+1].replace('оборот_', '').replace('_без_активации', ' без активации').replace('_', ' '))

res = pd.concat([msc, regs], axis=1)
#res = res.rename({27: 'Москва', 0: 'Регионы'}, axis=1)

figs = []
for i in range(0, 34, 2):
    fig = make_subplots(rows=1, cols=2, column_titles=['Москва', 'Регионы'], y_title='Оборот %')
    bar1 =px.bar(y=[res.iloc[i]['Москва'], res.iloc[i+1]['Москва']], x=[ccs[i], ccs[i+1]], text_auto='.4s')
    bar2 = px.bar(y=[res.iloc[i]['Регионы'], res.iloc[i+1]['Регионы']], x=[ccs[i], ccs[i+1]], text_auto='.4s')
    fig.add_trace(bar1.data[0], row=1, col=1)
    fig.add_trace(bar2.data[0], row=1, col=2)
    fig.update_traces(showlegend=False, textangle=0, marker=dict(color=['#F9B810', '#6C7074']))
    #fig.update_layout(barmode='relative')
    #fig.update_yaxes(title_text="Оборот %")
    figs.append(fig)
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#a6a6a6')
    fig.update_layout(width=1200, height=600, font=dict(size=14), paper_bgcolor='#F7FBFD', plot_bgcolor='#F7FBFD')
    #msc_fig = px.bar(y=[res.iloc[i]['Москва'], res.iloc[i+1]['Москва']], x=[ccs[i], ccs[i+1]])
    #regs_fig = px.bar(y=[res.iloc[i]['Регионы'], res.iloc[i+1]['Регионы']], x=[ccs[i], ccs[i+1]])
    #figs.append(msc_fig)
    #figs.append(regs_fig)
#figs[0].write_image('test.svg')
#for i in range(len(figs)): figs[i].write_image(f'./graphics1/{cats[i]}.svg')
for fig in figs: fig.show()

In [9]:
zero_count = 0
act1 = act.drop('Москва')
for column in act1.columns:
    cc = act1[column][act1[column] == 0].count()
    zero_count += cc
    print(f'{column}: {cc}')
print('Всего:', zero_count)

оборот_аптеки_с_активацией: 1
оборот_аптеки_без_активации: 1
оборот_рестораны_с_активацией: 1
оборот_рестораны_без_активации: 1
оборот_одежда_и_обувь_с_активацией: 0
оборот_одежда_и_обувь_без_активации: 0
оборот_автоуслуги_с_активацией: 4
оборот_автоуслуги_без_активации: 3
оборот_супермаркеты_с_активацией: 1
оборот_супермаркеты_без_активации: 2
оборот_такси_с_активацией: 5
оборот_такси_без_активации: 6
оборот_красота_с_активацией: 4
оборот_красота_без_активации: 3
оборот_развлечения_с_активацией: 2
оборот_развлечения_без_активации: 1
оборот_жд_билеты_с_активацией: 15
оборот_жд_билеты_без_активации: 14
оборот_образование_с_активацией: 12
оборот_образование_без_активации: 13
оборот_дом_и_ремонт_с_активацией: 18
оборот_дом_и_ремонт_без_активации: 15
оборот_спорттовары_с_активацией: 6
оборот_спорттовары_без_активации: 5
оборот_животные_с_активацией: 9
оборот_животные_без_активации: 7
оборот_цветы_с_активацией: 11
оборот_цветы_без_активации: 9
оборот_фастфуд_с_активацией: 40
оборот_фастфуд_

In [10]:
#import plotly.express as px

#data = ['14', '11', '10', '9', '8', '8', '8', '7', '6', '6', '5', '5', '5', '5', '4', '4', '4', '4', '4', '4', '4', '3', '3', '3', '3', '3', '3', '3', '3', '2', '2', '2', '1', '1', '1', '1']; data.reverse()
#data = [int(i) for i in data]

#X = list(range(len(data)))

#px.line(y=data, x=X)


In [11]:
#counts = df['регион_проживания'].value_counts() / 6
#import plotly.express as px

#Y = [counts['Москва'], counts['Санкт-Петербург'], counts[[c for c in counts.keys() if c not in ['Москва', 'Санкт-Петербург']]].sum()]
#X = ['Москва', 'Санкт-Петербург', 'Регионы']
#counts = pd.DataFrame({'Регион': X, 'Население': Y})

#px.pie(counts, values='Население', names='Регион')





#counts = df['регион_проживания'].value_counts() / 6
#import plotly.express as px
#import plotly.graph_objects as go

#Y = [counts['Москва'], counts['Санкт-Петербург'], counts[[c for c in counts.keys() if c not in ['Москва', 'Санкт-Петербург']]].sum()]
#X = ['Москва', 'Санкт-Петербург', 'Регионы']
#colors = ['gold', 'lightgreen', 'mediumturquoise']
#counts = pd.DataFrame({'Регион': X, 'Население': Y, 'colors': colors})

#fig = go.Figure(data=[go.Pie(labels=X, values=Y)])
#fig.update_traces(marker=dict(colors=colors))
#fig.show()





#px.pie(counts, values='Население', names='Регион', color='color')